In [ ]:
import os 
import re 
import json 
import string 
import numpy as np 
import tensorflow as tf 
from tensorflow import keras
from keras import layers 
from tokenizers import BertWordPieceTokenizer 
from transformers import BertTokenizer , TFBertModel , BertConfig 

max_len = 384 
configuration = BertConfig()


Bert Tokenizers 

In [ ]:
# save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
save_path = 'bert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

slow_tokenizer.save_pretrained(save_path)

# Load the fasst tokenizer from saved file 
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True) 

Load the data

In [ ]:
train_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json"
train_path = keras.utils.get_file('train.json', train_data_url)
eval_data_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
eval_path =  keras.utils.get_file("eval.json", eval_data_url)

Preprocessor data


In [ ]:
class SquadExample:
    def __init__(self, question , context , start_char_idx , answer_text, all_answers):
        super().__init__()
        self.question = question 
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocessor (self):
        context = self.context 
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx 

        # xóa đi các khoảng trắng trong câu hỏi và câu trả lời 
        context = ' '.join(str(context).split())
        question = ' '.join(str(question).split())
        answer = ' '.join(str(answer).split())

        # Tìm kiêm vị trí cuối cùng của câu trả lời 
        # bằng kí tự đầu tiên + độ dài câu trả lời 
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True 
            return 
        # Đánh dấu các chỉ mục ký tự trong ngữ cảnh có trong câu trả lời 
        # Đầu tiên biến các token có trong con text = 0
        is_char_in_ans = [0] * len(context)
        
        # Đánh các token có trong câu tl bằng 1 
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Thực hiện mã hóa thông báo context 
        tokenized_context = tokenizer.encode(context)

        # Tìm mã thông báo được tạo từ ký tự trả lời 
        # Khởi tạo 1 danh sách , danh sách này sẽ chứa vị trí của các token thuộc về câu trả lời 
        ans_token_idx =  []
        # lặp qua danh sách các offeets mã hóa thông báo context từ start đến end 
        # từ đó ta sẽ có được vị trí và số ký tự cho mỗi token
        for idx , (start , end) in enumerate(tokenized_context.offsets):
            # Trường hợp nếu tổng của ký tự cho câu trả lời của token # 0 
            # Tức là trong context có chứa câu trả lời 
            if sum(is_char_in_ans[start , end]) > 0:
                # Ta sẽ thêm vị trí cho token thuộc về câu trả lời dựa vào danh sách offsets 
                ans_token_idx.append(idx)

        # nếu như vị trí của token trong câu trả lừi = 0 tực khôngb không tồn tại câu tl 
        # ta bỏ qua 
        if len(ans_token_idx) == 0:
            self.skip = True 
            return 
        
        # Lấy ra vị trí bắt đầu và kết thúc của mỗi câu trả lời trong token
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Thực hiện mã hóa cho câu hỏi 
        tokenized_question = tokenizer.encode(question)

        # Khởi tạo đầu vào cho mô hình 
        # bằng token_context + token_question là những ma trận mã hóa số 
        input_ids = tokenized_context.ids + tokenized_question.ids[:1]
        # biến đổi thành các kiểu khác nhau cho cả 2 câu thành 0  và 1 
        token_type_ids = [0] * len(tokenized_context.ids) + [1] *len(
            tokenized_question.ids[1:]
        )
        # Khởi tạo mặt nạ attention 
        attention_mask = [1] * len(input_ids)

        # Tiến hành thêm đệm cho input , token_type , mask 
        # Tất cả các đệm này đêu là 0 
        # Độ dài của phần đệm = độ dài tối đa - độ dài của đầu vào mô hình 
        padding_length = max_len - len(input_ids)
        # trường hợp padding_length  > 0 tức là cần đệm 
        if padding_length > 0:
            input_ids = input_ids + ( [0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        # Ngược lại nếu padding_length < 0  tức là không cầm đệm ta bỏ qua 
        elif padding_length < 0:
            self.skip = True 
            return 
        

        # Trả về một loạt các danh sách input_ids , token_type_ids , attention_mask
        # start_token_idx . end_token_idx , tokenized_context.offsets 
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


with open(train_path)  as f:
    raw_train_data = json.load(f)

with open(eval_path) as f:
    raw_eval_data = json.load(f)

# paragraphs  đoạn văn 

#  Thực hiện hóa tách và biến đổi dữ liệu 
def create_squad_examples(raw_data):
    squad_examples = []
    # Duyệt qua các phần tử trong raw_data['data'] 
    # mỗi phần tử tượng trưng như một bài viết trong từ điển wiki 
    for item in raw_data['data']:
        # sau đó duyệt qua các đoạn văn bản có trong mỗi bài viết 
        for para in item['paragraphs']:
            # Lấy các đoạn văn bản mỗi đoạn văn bản sẽ chứa các chuỗi context. 
            context = para['context']
            # Tiếp tục duyệt qua một danh sách qa gồm các câu hỏi và trả lời 
            for qa in para['pas']:
                # lấy ra các question từ tập qas  và answer từ qas
                question = qa["question"]
                answer_text = qa["answers"][0]["text"] 
                # lấy ra danh sách các chuỗi câu trả lời đúng cho câu hỏi từ đoạn văn bản 
                # vì có nhiều cách trả lời khác nhau cho câu hỏi 
                all_answers = [_['text'] for _ in qa['answers']]
                # Lấy ra vị trí của kí tự đầu tiên thuộc câu trả lời 
                start_char_idx = qa['answers'][0]['answer_start']
                squad_eg = SquadExample(
                    question , context , start_char_idx , answer_text , all_answers
                )
                # Tiền xử lý 
                squad_eg.preprocessor()
                squad_examples.append(squad_eg)
    
    return squad_examples

# Xây dưngj lớp xử lý dữ liệu đầu vào 
def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item , key))
    
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])
    
    x =  [
        dataset_dict['input_ids'],
        dataset_dict['token_type_ids'],
        dataset_dict['attention_mask'],
    ]
    y = [dataset_dict['start_token_idx'] , dataset_dict['end_token_idx']]
    return x , y 

# Xây dữ liệu train và tets dựa trên raw_data đã tải trước đó 
train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

In [ ]:
def create_model():
    ## Bert model 
    encoder = TFBertModel.from_pretrained('bert-base-uncased')

    # QA Model 
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
     
    embedding = encoder(
        input_ids, token_type_ids , attention_mask 
    )[0]

    start_logits = layers.Dense(1  , name='start_logit' , use_bais=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)
    
    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs = [input_ids , token_type_ids , attention_mask],
        ouputs=[start_probs , end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(
        optimizer=optimizer, loss=[loss, loss]
    )
    return model



In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

Create Callbacks 

In [ ]:
def normalize_text(text): 
    # biến đổi văn bản thành chữ thường 
    text = text.lower()

    # Remove punctuations xóa ký tự đắc biệt 
    # Dầu tiên tạo một dannh sách chứa các ký tự đặc biệt 
    exclude = set(string.punctuation)
    # Sau đó tiến hành loại bỏ các ký tự đặc biệt 
    # bằng cách xây dựng lại 1 con text mới thay thế nó = ''
    text = "".join(_ for _ in text if _ not in exclude)

    # sử dụng re.sub để thay thế loạt chữ liệt kể = " "
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text

def normalize_text(text):
    text = text.lower()
    # Remove punctuations xóa dấu câu
    exclude = set(string.punctuation)
    # sau đó nối lại các ký tự trong văn bản và không thuộc tập exclude 
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles sử dụng biên dịch biểu thức 
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    # sử dụng re.sub để thay thế các ký tự =  " "  và trả về 1 text 
    text = re.sub(regex, ' ', text)

    # Xoa khoẳng trắng thừa 
    text =  " ".join(text.split())
    return text 

class ExactMatch(keras.callbacks.Callback):
    """
    Mỗi đối tượng `SquadExample` chứa độ lệch cấp độ ký tự cho mỗi mã thông báo
    trong đoạn đầu vào của nó.
    
    Chúng tôi sử dụng chúng để lấy lại khoảng văn bản tương ứng
    đến các mã thông báo giữa mã thông báo bắt đầu và kết thúc được dự đoán .
    
    Tất cả các câu trả lời thực tế cũng có trong mỗi đối tượng `SquadExample`.
    tính toán tỷ lệ phần trăm của các điểm dữ liệu mà khoảng văn bản thu được
    từ các dự đoán mô hình phù hợp với một trong những câu trả lời đúng sự thật.
        """
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self , epoch, logs=None):
        pred_start , pred_end = self.model.predict(self.x_eval)
        # Khởi tạo cuont để đếm số lượng câu trả lời chính xác 
        count = 0
        # Lọc qua những vi dụ trong tập dư liệu kiểm tra eval_squad_examples mà không
        # bị bỏ qua 
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        # Duyệt qua những cặp vị trí bắt đầu và kết thúc dự đoán cùng với những chỉ số tương ứng 
        for idx , (start , end) in enumerate(zip(pred_start , pred_end)):
            # Lấy ra ví dụ tương ứng trong tập dữ liệu kiểm tra đã lọc
            squad_eg = eval_examples_no_skip[idx]
            # lấy ra danh sách cac vị trí ký tự trong đoạn văn bản tương ứng với từng token
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            # Nếu vị trí bắt đầu lớn hơn hoặc bằng độ dài của danh sách offsets,
            #  phương thức này sẽ bỏ qua ví dụ này và tiếp tục vòng lặp
            # tức là dự đoán không thuộc offsets
            if start >=  len(offsets):
                continue
            # Nếu không, lấy ra vị trí ký tự bắt đầu của token bắt đầu 
            pred_char_start = offsets[start][0]
            # Nếu vị trí kết thúc nhỏ hơn độ dài của danh sách offsets, phương thức này sẽ lấy ra 
            # vị trí ký tự bắt đầu và kết thúc của câu trả lời trong văn bản ngữ cảnh 
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                # thực hiện khoanh vung cho câu tl
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else : 
            # Ngược lại ta lấy từ vị trí ký tự dự đoán đầu tiên đến hết câu . 
                pred_ans = squad_eg.context[pred_char_start:]
            
            # Chuẩn hóa câu trả lời dự đoán
            normalized_pred_ans = normalize_text(pred_ans)
            # tạo danh sách chuẩn hóa tất cả các nhãn thực 
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            
            # nếu như các nhãn dự đoán mà thuộc nhãn thực thì tăng count lên 1 
            if normalized_pred_ans in normalized_true_ans:
                count += 1 
        # Sau khi duyệt hết các ví dụ kiểm tra, phương thức này sẽ tính toán độ chính xác của mô hình
        # bằng cách chia số lượng câu trả lời chính xác cho số lượng ví dụ kiểm tra
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")



Train and Evaluate

In [ ]:
exact_match_callback = ExactMatch(x_eval, y_eval)
model.fit(
    x_train,
    y_train,
    epochs=1,  # For demonstration, 3 epochs are recommended
    verbose=2,
    batch_size=64,
    callbacks=[exact_match_callback],
)